In [11]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy import linalg
import algoritmo_ncm as ncm

In [12]:
ncm.sm_ncm(x_input,g_input,iter,tol)

AttributeError: module 'algoritmo_ncm' has no attribute 'sm_ncm'